# Extracting a single channel

Import the various 'lsdviztools' and 'lsdttparamselector' modules.

In [1]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

## Get the DEM

Grab the DEM from opentopography.org

In [2]:
SB_DEM = bmt.ot_scraper(source = "SRTM30",longitude_W = -120.464655, longitude_E = -120.254214, latitude_S = 34.440538, latitude_N = 34.610770,prefix = "CP")
SB_DEM.print_parameters()

The source is: SRTM30
The west longitude is: -120.464655
The east longitude is: -120.254214
The south latitude is: 34.61077
The north latitude is: 34.440538
The path is: ./
The prefix is: CP


In [3]:
SB_DEM.download_pythonic()

I am going to download the following for you:
https://portal.opentopography.org/otr/getdem?demtype=SRTMGL1&south=34.440538&north=34.61077&west=-120.464655&east=-120.254214&outputFormat=GTiff
This might take a little while, depending on the size of the file. 
Finished downloading


Convert to UTM coordinates and ENVI bil format for *LSDTopoTools*

In [4]:
SB_DEM.to_UTM_pythonic()
RasterFile = "CP_SRTM30_UTM.tif"
DataDirectory = "./"
gio.convert2bil(DataDirectory, RasterFile,minimum_elevation=0)

The projections is:
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
And some extra projection information strings:
None
WGS 84
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 758, 'height': 613, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777778146, 0.0, -120.46486111110323,
       0.0, -0.0002777777777778146, 34.61097222221886)}
res tuple is:
(30.0, 30.0)
The destination CRS is: EPSG:32610
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 661, 'height': 647, 'count': 1, 'crs': CRS.from_epsg(32610), 'transform': Affine(30.0, 0.0, 732452.2754693646,
       0.0, -30.0, 3833330.7356094)}
Removing pixels with an elevation less than 0


# Get the point from which to extract the channel

In [5]:
import pandas as pd

In [6]:
d = {'id': [0], 'latitude': [34.550732], 'longitude': [-120.287266]}
df = pd.DataFrame(data=d)

Print this to a csv file

In [7]:
df.to_csv("channel_source.csv",index=False)

# Set up parameters for an *lsdtopotools* run

In [8]:
lsdtt_parameters = {"write_hillshade" : "true", 
                    "extract_single_channel" : "true", 
                    "channel_source_fname" : "channel_source.csv", 
                    "print_dinf_drainage_area_raster" : "true",
                    "convert_csv_to_geojson" : "true"}

Create a driver object

In [9]:
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "CP_SRTM30_UTM",
                                 write_prefix= "CP_SRTM30_UTM",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

The command line tool is: lsdtt-basic-metrics
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: CP_SRTM30_UTM
The write prefix is: CP_SRTM30_UTM
The parameter dictionary is:
{'write_hillshade': 'true', 'extract_single_channel': 'true', 'channel_source_fname': 'channel_source.csv', 'print_dinf_drainage_area_raster': 'true', 'convert_csv_to_geojson': 'true'}


Run *lsdtopotools*

In [10]:
lsdtt_drive.run_lsdtt_command_line_tool()

Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.
